In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
df = pd.read_csv('C:/Users/Olga/Downloads/archive22/train_v2_drcat_02.csv')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

In [4]:
# Кодування міток класів
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [5]:
# Токенізація тексту
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [6]:
# Перетворення тексту в послідовності чисел
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [7]:
# Доповнення послідовностей до одного розміру
max_sequence_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')


In [8]:
# Побудова моделі
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [9]:
# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Навчання моделі
epochs = 5
batch_size = 32
model.fit(X_train_padded, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/5


898/898 [==============================] - 29s 31ms/step - loss: 0.1823 - accuracy: 0.9302 - val_loss: 0.0882 - val_accuracy: 0.9699
Epoch 2/5
898/898 [==============================] - 28s 31ms/step - loss: 0.0622 - accuracy: 0.9810 - val_loss: 0.0547 - val_accuracy: 0.9824
Epoch 3/5
898/898 [==============================] - 29s 32ms/step - loss: 0.0314 - accuracy: 0.9904 - val_loss: 0.0802 - val_accuracy: 0.9802
Epoch 4/5
898/898 [==============================] - 30s 34ms/step - loss: 0.0284 - accuracy: 0.9912 - val_loss: 0.0551 - val_accuracy: 0.9833
Epoch 5/5
898/898 [==============================] - 28s 31ms/step - loss: 0.0184 - accuracy: 0.9948 - val_loss: 0.0539 - val_accuracy: 0.9840


In [13]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(X_test_padded, y_test_encoded)
print(f'Точність на тестових даних: {test_acc}')

281/281 [==============================] - 3s 10ms/step - loss: 0.0573 - accuracy: 0.9832
Точність на тестових даних: 0.9831736087799072
